### How to Remove and Add Back a Constraint
In this notebook, we will learn how to remove a constraint from an existing model and how to add it back later.

To better understand the problem, you can view it in the corresponding notebook available on my GitHub account. https://github.com/baraa90m/Mathematical-Optimization/blob/main/Gurobi%20Optimization/Tutorials/milp_RAP_problem_002.ipynb


\begin{aligned}
&\text{Maximize} \\
&\quad 53\, \text{assign}[\text{Carlos}, \text{Tester}] + 27\, \text{assign}[\text{Carlos}, \text{JavaDeveloper}] \\
&\quad + 13\, \text{assign}[\text{Carlos}, \text{Architect}] \\
&\quad + 80\, \text{assign}[\text{Joe}, \text{Tester}] + 47\, \text{assign}[\text{Joe}, \text{JavaDeveloper}] \\
&\quad + 67\, \text{assign}[\text{Joe}, \text{Architect}] \\
&\quad + 53\, \text{assign}[\text{Monika}, \text{Tester}] + 73\, \text{assign}[\text{Monika}, \text{JavaDeveloper}] \\
&\quad + 47\, \text{assign}[\text{Monika}, \text{Architect}] \\[2ex]
&\text{Subject To} \\[1ex]
&\quad \text{job}[\text{Tester}]: \\
&\qquad \text{assign}[\text{Carlos}, \text{Tester}] + \text{assign}[\text{Joe}, \text{Tester}] + \text{assign}[\text{Monika}, \text{Tester}] = 1 \\[1ex]
&\quad \text{job}[\text{JavaDeveloper}]: \\
&\qquad \text{assign}[\text{Carlos}, \text{JavaDeveloper}] + \text{assign}[\text{Joe}, \text{JavaDeveloper}] \\
&\qquad + \text{assign}[\text{Monika}, \text{JavaDeveloper}] = 1 \\[1ex]
&\quad \text{job}[\text{Architect}]: \\
&\qquad \text{assign}[\text{Carlos}, \text{Architect}] + \text{assign}[\text{Joe}, \text{Architect}] \\
&\qquad + \text{assign}[\text{Monika}, \text{Architect}] = 1 \\[1ex]
&\quad \text{resource}[\text{Carlos}]: \\
&\qquad \text{assign}[\text{Carlos}, \text{Tester}] + \text{assign}[\text{Carlos}, \text{JavaDeveloper}] \\
&\qquad + \text{assign}[\text{Carlos}, \text{Architect}] \leq 1 \\[1ex]
&\quad \text{resource}[\text{Joe}]: \\
&\qquad \text{assign}[\text{Joe}, \text{Tester}] + \text{assign}[\text{Joe}, \text{JavaDeveloper}] \\
&\qquad + \text{assign}[\text{Joe}, \text{Architect}] \leq 1 \\[1ex]
&\quad \text{resource}[\text{Monika}]: \\
&\qquad \text{assign}[\text{Monika}, \text{Tester}] + \text{assign}[\text{Monika}, \text{JavaDeveloper}] \\
&\qquad + \text{assign}[\text{Monika}, \text{Architect}] \leq 1
\end{aligned}


In [1]:
import gurobipy as gp
from gurobipy import GRB

#### Import the saved model:
The problem we are working with has already been solved and saved in models/RAP.lp.

We now simply need to load it and continue by removing and re-adding a constraint.

In [4]:
m = gp.read('models/RAP.lp')

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22
Read LP format model from file models/RAP.lp
Reading time = 0.00 seconds
: 6 rows, 9 columns, 18 nonzeros


In [6]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [8]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


#### Removing the constraint for job[Tester]:

In [10]:
c = m.getConstrs()[0]
c

<gurobi.Constr job[Tester]>

In [12]:
lhs, sense, rhs, name = m.getRow(c), c.Sense, c.RHS, c.ConstrName
m.remove(c)

In [14]:
# Show the current state
m.update()
m.printStats()

Statistics for unnamed model:
  Problem type                : LP
  Linear constraint matrix    : 5 rows, 9 columns, 15 nonzeros
  Variable types              : 9 continuous, 0 integer (0 binary)
  Matrix range                : [1e+00, 1e+00]
  Objective range             : [1e+01, 8e+01]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [1e+00, 1e+00]


In [16]:
m.getConstrs()

[<gurobi.Constr job[JavaDeveloper]>,
 <gurobi.Constr job[Architect]>,
 <gurobi.Constr resource[Carlos]>,
 <gurobi.Constr resource[Joe]>,
 <gurobi.Constr resource[Monika]>]

We notice  that the constraint for Tester-job is removed.

#### Re-adding the Removed Constraint:

In [24]:
c = m.addConstr(lhs == rhs, name)

In [26]:
# Show the current state
m.update()
m.printStats()

Statistics for unnamed model:
  Problem type                : LP
  Linear constraint matrix    : 6 rows, 9 columns, 18 nonzeros
  Variable types              : 9 continuous, 0 integer (0 binary)
  Matrix range                : [1e+00, 1e+00]
  Objective range             : [1e+01, 8e+01]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [1e+00, 1e+00]


In [28]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
